In [ ]:
# install datasets
!pip install folium==0.2.1
!pip install datasets

# Make sure that we have a recent version of pyarrow in the session before we continue - otherwise reboot Colab to activate it
import pyarrow
if int(pyarrow.__version__.split('.')[1]) < 16 and int(pyarrow.__version__.split('.')[0]) == 0:
    import os
    os.kill(os.getpid(), 9)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 8.0 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=f67966214e69e316f4fd40b209faa71c47aa5b7d94f1fe22a12648efad9e3e4f
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 13.4 MB/s 
     |████████████████████████████████| 1.1 MB 61.9 MB/s 
     |████████████████████████████████| 101 kB 9.2 MB/s 
     |████████████████████████████████| 212 kB 54.3 MB/s 
     |████████████████████████████████| 140 kB 71.9 MB/s 
     |████████████████

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 32.2 MB/s 
     |████████████████████████████████| 6.6 MB 63.5 MB/s 


In [ ]:
import torch

from datasets import load_dataset, list_datasets, list_metrics
from pprint import pprint
from tqdm import tqdm
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [ ]:
datasets = list_datasets()
print(len(datasets), datasets[0:10])

6984 ['acronym_identification', 'ade_corpus_v2', 'adversarial_qa', 'aeslc', 'afrikaans_ner_corpus', 'ag_news', 'ai2_arc', 'air_dialogue', 'ajgt_twitter_ar', 'allegro_reviews']


In [ ]:
glue_dataset = list_datasets(with_details=True)[datasets.index('glue')]

In [ ]:
pprint(glue_dataset)

DatasetInfo: {
	id: glue
	sha: 9338f7b671827df886678df2bdd7cc7b4f36dffd
	lastModified: 2022-07-01T11:52:07.000Z
	tags: ['annotations_creators:unknown', 'language_creators:unknown', 'language:en', 'license:cc-by-4.0', 'multilinguality:monolingual', 'size_categories:10K<n<100K', 'source_datasets:unknown', 'task_categories:text-classification', 'task_ids:acceptability-classification', 'task_ids:natural-language-inference', 'task_ids:semantic-similarity-scoring', 'task_ids:sentiment-classification', 'task_ids:text-classification-other-coreference-nli', 'task_ids:text-classification-other-paraphrase-identification', 'task_ids:text-classification-other-qa-nli', 'task_ids:text-scoring', 'pretty_name:GLUE (General Language Understanding Evaluation benchmark)', 'configs:ax', 'configs:cola', 'configs:mnli', 'configs:mnli_matched', 'configs:mnli_mismatched', 'configs:mrpc', 'configs:qnli', 'configs:qqp', 'configs:rte', 'configs:sst2', 'configs:stsb', 'configs:wnli']
	private: False
	author: None


## GLUE: A MULTI-TASK BENCHMARK AND ANALYSIS PLATFORM FOR NATURAL LANGUAGE UNDERSTANDING

**GLUE** contains 11 tasks including MRPC, STS, QQP, and several NLI tasks. More details are available on https://gluebenchmark.com/tasks.

**MRPC (Microsoft Research Paraphrase Corpus):** https://www.microsoft.com/en-us/download/details.aspx?id=52398

5800 pairs of sentences have been extracted from news sources on the web, along with human annotations indicating whether each pair captures a paraphrase/semantic equivalence relationship.

In [ ]:
train_dataset = load_dataset('glue', 'mrpc', split='train')
test_dataset = load_dataset('glue', 'mrpc', split='test')

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [ ]:
def encode(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding='max_length')

train_dataset = train_dataset.map(encode, batched=True)
test_dataset = test_dataset.map(encode, batched=True)
print(train_dataset[0])

Parameter 'function'=<function encode at 0x7f8a813ce200> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', 'label': 1, 'idx': 0, 'input_ids': [101, 7277, 2180, 5303, 4806, 1117, 1711, 117, 2292, 1119, 1270, 107, 1103, 7737, 107, 117, 1104, 9938, 4267, 12223, 21811, 1117, 2554, 119, 102, 11336, 6732, 3384, 1106, 1140, 1112, 1178, 107, 1103, 7737, 107, 117, 7277, 2180, 5303, 4806, 1117, 1711, 1104, 9938, 4267, 12223, 21811, 1117, 2554, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
print(train_dataset[0].keys(), train_dataset[0]['label'])

dict_keys(['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask']) 1


Let's use a BERT model for [classification](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertForSequenceClassification).

Bert Model transformer with a sequence classification/regression head on top (a linear layer on top of the pooled output)


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
train_dataset = train_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)
train_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16)

  0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
!mkdir drive/MyDrive/Workshops/ACM\ Summer\ School/checkpoints

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)

In [ ]:
model.train().to(device)

In [ ]:
for epoch in range(2):
    for i, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if i % 10 == 0:
            print(f"loss: {loss}")

    if epoch % 5 == 0:
        torch.save(model, 'drive/MyDrive/Workshops/ACM Summer School/checkpoints/model.pt')

  0%|          | 1/230 [00:01<05:31,  1.45s/it]

loss: 0.7185303568840027


  5%|▍         | 11/230 [00:15<05:06,  1.40s/it]

loss: 0.38467368483543396


  9%|▉         | 21/230 [00:29<04:54,  1.41s/it]

...

 92%|█████████▏| 211/230 [05:17<00:28,  1.51s/it]

loss: 0.3816697299480438


 96%|█████████▌| 221/230 [05:32<00:13,  1.51s/it]

loss: 0.38189974427223206


100%|██████████| 230/230 [05:45<00:00,  1.50s/it]


In [ ]:
torch.save(model, 'drive/MyDrive/Workshops/ACM Summer School/checkpoints/model.pt')

## A copy of the model can be downloaded from here: https://drive.google.com/file/d/1-2QHPLPKvmdcwDtl2m9yE6V3KlY0wl-m/view?usp=sharing

In [ ]:
model = torch.load('drive/MyDrive/Workshops/ACM Summer School/checkpoints/model.pt')

model.train()

for epoch in range(1):
    for i, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if i % 10 == 0:
            print(f"loss: {loss}")
    torch.save(model, 'drive/MyDrive/Workshops/ACM Summer School/checkpoints/model.pt')

  0%|          | 1/230 [00:01<05:04,  1.33s/it]

loss: 0.46878889203071594


  5%|▍         | 11/230 [00:14<04:46,  1.31s/it]

loss: 0.16178524494171143


  9%|▉         | 21/230 [00:27<04:36,  1.32s/it]

...

 92%|█████████▏| 211/230 [04:58<00:27,  1.44s/it]

loss: 0.12079475075006485


 96%|█████████▌| 221/230 [05:12<00:12,  1.44s/it]

loss: 0.2723121643066406


100%|██████████| 230/230 [05:24<00:00,  1.41s/it]


In [ ]:
test_dataset = test_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)
test_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=16)

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
model.eval()

all_ground_truth = []
all_predictions = []

with torch.no_grad():
    for i, batch in enumerate(tqdm(test_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        logits = model(**batch)[1]
        predicted_class_ids = logits.argmax(dim=-1)
        all_ground_truth += batch['labels'].cpu().detach().numpy().tolist()
        all_predictions += predicted_class_ids.cpu().detach().numpy().tolist()

100%|██████████| 108/108 [01:00<00:00,  1.78it/s]


# New huggingface library **evaluate**
https://github.com/huggingface/evaluate

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 2.6 MB/s 


In [ ]:
import evaluate

In [ ]:
accuracy_metric = evaluate.load("accuracy")
results = accuracy_metric.compute(references=all_ground_truth, predictions=all_predictions)
print(results)

{'accuracy': 0.7791304347826087}


References:
1. GLUE https://openreview.net/pdf?id=rJ4km2R5t7
2. https://huggingface.co/docs/datasets/quickstart